In [ ]:
import numpy as np
import os
import matplotlib.pyplot as plt
import matplotlib
from matplotlib.ticker import MaxNLocator
%matplotlib inline
plt.ion();
import utils_intensity_map as uim
import utils_deck_generation as idg
import netcdf_read_write as nrw
import training_data_generation as tdg
import utils_healpy as uhp
import utils_optimizers as uopt

In [ ]:
diag_dir = "../Data/test_run"

sys_params = tdg.define_system_params(diag_dir)
sys_params["plot_file_type"] = ".png"

aspect_ratio = (5, 4)

In [ ]:
def fitness_function(dataset, opt_params):
    target_rms = opt_params["fitness_desired_rms"]
    norm_factor = opt_params["fitness_norm_factor"]
    number_of_timesteps = np.shape(dataset["rms"][:,:])[1]

    rms = np.sqrt(np.sum(dataset["rms"][:,:]**2, axis=1) / float(number_of_timesteps))
    avg_flux = np.sqrt(np.sum(dataset["avg_flux"][:,:]**2, axis=1) / float(number_of_timesteps))
    if opt_params["run_plasma_profile"]:
        target_flux = opt_params["fitness_desired_pressure_mbar"]
        indices = np.where(np.array(avg_flux) > opt_params["fitness_limit_broken_pressure_mbar"])[0]
        print("Fitness function detects broken runs: ", indices)
        avg_flux[indices] = 0.0
    else:
        target_flux = opt_params["fitness_desired_power_per_steradian"]

    maxi_func = np.exp(-(rms/target_rms) + (avg_flux / target_flux) ** 0.25) * (avg_flux / target_flux)**0.01 * norm_factor
    #maxi_func = np.exp(-(rms/target_rms) + (avg_flux / target_flux) ** 0.25) * (avg_flux / target_flux) * norm_factor
    return maxi_func

def fitness_function_time_dependant(dataset, opt_params):
    target_rms = opt_params["fitness_desired_rms"]
    norm_factor = opt_params["fitness_norm_factor"]
    number_of_timesteps = np.shape(dataset["rms"][:,:])[1]

    rms = dataset["rms"][:,:]
    avg_flux = dataset["avg_flux"][:,:]
    if opt_params["run_plasma_profile"]:
        target_flux = opt_params["fitness_desired_pressure_mbar"]
        indices = np.where(np.array(avg_flux) > opt_params["fitness_limit_broken_pressure_mbar"])[0]
        print("Fitness function detects broken runs: ", indices)
        avg_flux[indices] = 0.0
        indices = np.where(np.array(avg_flux) < opt_params["fitness_limit_broken_pressure_mbar"] * 0.001)[0]
        print("Fitness function detects broken runs: ", indices)
    else:
        target_flux = opt_params["fitness_desired_power_per_steradian"]

    maxi_func = np.exp(-(rms/target_rms) + (avg_flux / target_flux) ** 0.25) * (avg_flux / target_flux)**0.01 * norm_factor
    return maxi_func

In [ ]:
dataset, dataset_params, deck_gen_params, facility_spec = idg.load_data_dicts_from_file(sys_params)
opt_params = uopt.define_optimizer_parameters(diag_dir, 0, 0, dataset_params, facility_spec, sys_params)

initial_dataset = dataset_params["num_examples"] #16#36

fitness_temporal = fitness_function_time_dependant(dataset, opt_params)
fitness_overall = fitness_function(dataset, opt_params)

labels = [None] * dataset_params["num_examples"]
for iconfig in range(dataset_params["num_examples"]):
    labels[iconfig] = r"SG={:.2f} and $R_b/R_t$={:.2f} ".format(deck_gen_params["beamspot_order"][iconfig,0], deck_gen_params["beamspot_major_radius"][iconfig,0]/dataset_params['target_radius'])

In [ ]:
ind_max_fitness = np.argmax(fitness_overall)
print(ind_max_fitness)
print(fitness_overall[ind_max_fitness])

#"""
param1 = deck_gen_params["beamspot_order"][:,0]
label1 = "Super gaussian order"
param2 = deck_gen_params["beamspot_major_radius"][:,0]/dataset_params['target_radius']
label2 = r"Beam to target ratio ($R_b/R_t$)"
semilogy_bool = False
"""
param1 = deck_gen_params["bandwidth_num_spectral_lines"]
label1 = "Number spectral lines"
param2 = deck_gen_params["bandwidth_percentage_width"][:,0]
label2 = r"Bandwidth ($\delta \lambda / \lambda$)"
semilogy_bool = True
#"""

In [ ]:
fig1, ax1 = plt.subplots(figsize=aspect_ratio, dpi=400)

cmap = ax1.scatter(param1, param2, c=fitness_overall,s=100)
fig1.colorbar(cmap, ax=ax1, label="Overall fitness")
if semilogy_bool:
    ax1.semilogy(param1[ind_max_fitness],param2[ind_max_fitness],"rx")
else:
    ax1.plot(param1[ind_max_fitness],param2[ind_max_fitness],"rx")

ax1.set_xlabel(label1)
ax1.set_ylabel(label2)
plt.savefig(diag_dir+"/scan_results_overall_fitness_points" + sys_params["plot_file_type"], dpi=300, bbox_inches='tight')

In [ ]:
number_of_timesteps = np.shape(dataset["rms"])[1]

rms_overall = np.sqrt(np.sum(dataset["rms"]**2, axis=1) / float(number_of_timesteps))
avg_flux_overall = np.sqrt(np.sum(dataset["avg_flux"]**2, axis=1) / float(number_of_timesteps))
print(np.shape(rms_overall), np.shape(rms_overall))

num_samples_per_param = int(np.sqrt(initial_dataset)) #int(np.sqrt(dataset_params["num_examples"]))
rms_overall_reshaped = np.reshape(rms_overall[:initial_dataset],(num_samples_per_param,num_samples_per_param))
rms_temporal_reshaped = np.reshape(dataset["rms"][:initial_dataset],(num_samples_per_param,num_samples_per_param,-1))
avg_flux_overall_reshaped = np.reshape(avg_flux_overall[:initial_dataset],(num_samples_per_param,num_samples_per_param))
avg_flux_temporal_reshaped = np.reshape(dataset["avg_flux"][:initial_dataset],(num_samples_per_param,num_samples_per_param,-1))
fitness_overall_reshaped = np.reshape(fitness_overall[:initial_dataset],(num_samples_per_param,num_samples_per_param))
fitness_temporal_reshaped = np.reshape(fitness_temporal[:initial_dataset],(num_samples_per_param,num_samples_per_param,-1))
print(np.shape(fitness_temporal_reshaped), np.shape(avg_flux_temporal_reshaped))
print(np.shape(dataset["input_parameters"]))

#X = np.reshape(dataset["input_parameters"][:,0],(num_samples_per_param,num_samples_per_param))
#Y = np.reshape(dataset["input_parameters"][:,1],(num_samples_per_param,num_samples_per_param))

X = np.reshape(param1[:initial_dataset],(num_samples_per_param,num_samples_per_param))
Y = np.reshape(param2[:initial_dataset],(num_samples_per_param,num_samples_per_param))
print(np.shape(X), np.shape(Y))
#print(rms_overall)

In [ ]:
fig1, ax1 = plt.subplots(figsize=aspect_ratio, dpi=400)
fig2, ax2 = plt.subplots(figsize=aspect_ratio, dpi=400)
fig3, ax3 = plt.subplots(figsize=aspect_ratio, dpi=400)

cmap = ax1.pcolormesh(X, Y, fitness_overall_reshaped)
fig1.colorbar(cmap, ax=ax1, label="Overall fitness")

cmap = ax2.pcolormesh(X, Y, rms_overall_reshaped * 100., norm=matplotlib.colors.LogNorm(vmax=2))
fig2.colorbar(cmap, ax=ax2, label="Overall rms (%)")

cmap = ax3.pcolormesh(X, Y, avg_flux_overall_reshaped)
fig3.colorbar(cmap, ax=ax3, label="Overall ablation pressure (Mbars)")

if semilogy_bool:
    ax1.semilogy(param1[ind_max_fitness],param2[ind_max_fitness],"rx")
    ax2.semilogy(param1[ind_max_fitness],param2[ind_max_fitness],"rx")
    ax3.semilogy(param1[ind_max_fitness],param2[ind_max_fitness],"rx")
else:
    ax1.plot(param1[ind_max_fitness],param2[ind_max_fitness],"rx")
    ax2.plot(param1[ind_max_fitness],param2[ind_max_fitness],"rx")
    ax3.plot(param1[ind_max_fitness],param2[ind_max_fitness],"rx")


ax1.set_xlabel(label1)
ax1.set_ylabel(label2)
ax2.set_xlabel(label1)
ax2.set_ylabel(label2)
ax3.set_xlabel(label1)
ax3.set_ylabel(label2)
fig1.savefig(diag_dir+"/scan_results_overall_fitness_grid" + sys_params["plot_file_type"], dpi=300, bbox_inches='tight')
fig2.savefig(diag_dir+"/scan_results_overall_rms_grid" + sys_params["plot_file_type"], dpi=300, bbox_inches='tight')
fig3.savefig(diag_dir+"/scan_results_overall_pabl_grid" + sys_params["plot_file_type"], dpi=300, bbox_inches='tight')

In [ ]:
eval_time_ind = 0
ind_max_fitness_temp = np.argmax(fitness_temporal[:,eval_time_ind])
print(ind_max_fitness_temp)
print(fitness_temporal[ind_max_fitness_temp,eval_time_ind])
print(dataset["avg_flux"][ind_max_fitness_temp,eval_time_ind])
print(dataset["rms"][ind_max_fitness_temp,eval_time_ind])

"""
view_index = 33
print(view_index)
print(fitness_temporal[view_index,eval_time_ind])
print(dataset["avg_flux"][view_index,eval_time_ind])
print(dataset["rms"][view_index,eval_time_ind])
"""

fig1, ax1 = plt.subplots(figsize=aspect_ratio, dpi=400)
fig2, ax2 = plt.subplots(figsize=aspect_ratio, dpi=400)
fig3, ax3 = plt.subplots(figsize=aspect_ratio, dpi=400)

cmap = ax1.pcolormesh(X, Y, fitness_temporal_reshaped[:,:,eval_time_ind])
fig1.colorbar(cmap, ax=ax1, label="Time dependant fitness")

rms_temporal_reshaped = np.reshape(dataset["rms"][:initial_dataset],(num_samples_per_param,num_samples_per_param,-1))
cmap = ax2.pcolormesh(X, Y, rms_temporal_reshaped[:,:,eval_time_ind] * 100., norm=matplotlib.colors.LogNorm(vmax=2))
fig2.colorbar(cmap, ax=ax2, label="Time dependant rms (%)")

cmap = ax3.pcolormesh(X, Y, avg_flux_temporal_reshaped[:,:,eval_time_ind])
fig3.colorbar(cmap, ax=ax3, label="Time dependant ablation pressure (Mbars)")

if semilogy_bool:
    ax1.semilogy(param1[ind_max_fitness],param2[ind_max_fitness],"rx")
    ax1.semilogy(param1[ind_max_fitness_temp],param2[ind_max_fitness_temp],"ro")
    ax2.semilogy(param1[ind_max_fitness],param2[ind_max_fitness],"rx")
    ax2.semilogy(param1[ind_max_fitness_temp],param2[ind_max_fitness_temp],"ro")
    ax3.semilogy(param1[ind_max_fitness],param2[ind_max_fitness],"rx")
    ax3.semilogy(param1[ind_max_fitness_temp],param2[ind_max_fitness_temp],"ro")
else:
    ax1.plot(param1[ind_max_fitness],param2[ind_max_fitness],"rx")
    ax1.plot(param1[ind_max_fitness_temp],param2[ind_max_fitness_temp],"ro")
    ax2.plot(param1[ind_max_fitness],param2[ind_max_fitness],"rx")
    ax2.plot(param1[ind_max_fitness_temp],param2[ind_max_fitness_temp],"ro")
    ax3.plot(param1[ind_max_fitness],param2[ind_max_fitness],"rx")
    ax3.plot(param1[ind_max_fitness_temp],param2[ind_max_fitness_temp],"ro")

ax1.set_xlabel(label1)
ax1.set_ylabel(label2)
ax2.set_xlabel(label1)
ax2.set_ylabel(label2)
ax3.set_xlabel(label1)
ax3.set_ylabel(label2)
fig1.savefig(diag_dir+"/scan_results_tind_"+str(eval_time_ind)+"_fitness_grid" + sys_params["plot_file_type"], dpi=300, bbox_inches='tight')
fig2.savefig(diag_dir+"/scan_results_tind_"+str(eval_time_ind)+"_rms_grid" + sys_params["plot_file_type"], dpi=300, bbox_inches='tight')
fig3.savefig(diag_dir+"/scan_results_tind_"+str(eval_time_ind)+"_pabl_grid" + sys_params["plot_file_type"], dpi=300, bbox_inches='tight')

In [ ]:
ind_max_fitness_temp = np.argmax(fitness_temporal, axis=0)
print(ind_max_fitness, ind_max_fitness_temp)
print(np.argmax(np.sqrt((fitness_temporal[:,2]**2+fitness_temporal[:,3]**2)/2.0)))

fig1, ax1 = plt.subplots(figsize=aspect_ratio, dpi=400)
fig2, ax2 = plt.subplots(figsize=aspect_ratio, dpi=400)
fig3, ax3 = plt.subplots(figsize=aspect_ratio, dpi=400)

for ind in ind_max_fitness_temp:
    ax1.semilogy(dataset_params["plasma_profile_times"], fitness_temporal[ind,:], "x:", label=labels[ind])
    ax2.semilogy(dataset_params["plasma_profile_times"],dataset["rms"][ind,:] * 100, "x:", label=labels[ind])
    #ax2.semilogy(dataset_params["plasma_profile_times"],dataset["rms"][ind,:] * 100, ".", label=labels[ind])
    ax3.plot(dataset_params["plasma_profile_times"][:],dataset["avg_flux"][ind,:], "x:", label=labels[ind])
    #ax3.plot(dataset_params["plasma_profile_times"][:],dataset["avg_flux"][ind,:], ".", label=labels[ind])

xlims = [0.,30.0]
ylims = [0.01,10.0]

#ax1.set_xlim(xlims)
#ax1.set_ylim(ylims)
ax1.set_xlabel("Time (ns)")
ax1.set_ylabel("Fitness time dependant")
ax1.legend(loc="best")

xlims = [0.,30.0]
ylims = [0.,100.0]

#ax2.set_xlim(xlims)
#ax1.set_ylim(ylims)
ax2.set_xlabel("Time (ns)")
ax2.set_ylabel("RMS ablation pressure (%)")
ax2.legend(loc="best")
#fig1.savefig(root+"/"+plot_dir + "/" + 'hydro_equivalent_intensity.png', bbox_inches='tight')

#ax3.set_xlim(xlims)
ax3.set_xlabel("Time (ns)")
ax3.set_ylabel("Ablation pressure (Mbars)")
ax3.legend(loc="best")
#fig2.savefig(root+"/"+plot_dir + "/" + 'hydro_equivalent_ablation_pressure_estimate.png', bbox_inches='tight')

fig1.savefig(diag_dir+"/time_dependant_fitness" + sys_params["plot_file_type"], dpi=300, bbox_inches='tight')
fig2.savefig(diag_dir+"/time_dependant_rms" + sys_params["plot_file_type"], dpi=300, bbox_inches='tight')
fig3.savefig(diag_dir+"/time_dependant_pabl" + sys_params["plot_file_type"], dpi=300, bbox_inches='tight')

In [ ]:

fig2, ax2 = plt.subplots(figsize=aspect_ratio, dpi=400)

target_rms = opt_params["fitness_desired_rms"]
norm_factor = opt_params["fitness_norm_factor"]
if (np.mean(avg_flux_overall)>1.e6):
    target_flux = opt_params["fitness_desired_power_per_steradian"] * 3.0
else:
    target_flux = opt_params["fitness_desired_pressure_mbar"]

ax2.plot(fitness_overall, label=labels)
ax2.plot(2.-rms_overall / target_rms, "r:", label=labels)
ax2.plot(avg_flux_overall / target_flux, "r", label=labels)

ylims = [0,2.5]

ax2.set_ylim(ylims)
ax2.set_xlabel("Configuration number")
ax2.set_ylabel("Fitness (rms of all times)")
#ax2.legend(loc="best")


# Load Multi data

In [ ]:
import utils_multi as um
n_351 = um.critical_density()

In [ ]:
multi_filename = "multi_data"
input_filename = "multi_output.txt"
output_filename = "multi2ifriit.nc"

input_filename = "multi_input.txt"
output_filename = "multi_output.bin"
multi_data = {}

multi_data = um.multi_read_bin(diag_dir+"/"+multi_filename+"/"+output_filename, multi_data)
multi_data = um.read_inputs(diag_dir+"/"+multi_filename+"/"+input_filename, multi_data)
print("Number time outputs: ", np.shape(multi_data["TIME"])[0])
print("Number radial outputs: ", np.shape(multi_data["CMC"])[1])
print("Number fuel cells: ", multi_data["fuel_boundary"])

multi_data = um.multi_data_units(multi_data)
multi_data = um.multi_find_interfaces(multi_data)
multi_data = um.multi_critical_surface(multi_data, n_351)
multi_data = um.define_capsule(multi_data)
multi_data = um.multi_mean_laser_dep_radius(multi_data)
multi_data = um.define_implosion_velocity(multi_data)
multi_data = um.adiabat(multi_data)
multi_data = um.calc_rhor(multi_data)
multi_data = um.lawson_criteria(multi_data)
print("Number time outputs: ", np.shape(multi_data["TIME"])[0])
print("Number radial outputs: ", np.shape(multi_data["CMC"])[1])
um.multi_printout(multi_data)

In [ ]:
aspect_ratio = (4,4)
list_colours = [
                "tab:blue",
                "tab:purple",
                "tab:green",
                "tab:red",
                "tab:cyan",
                "tab:pink",
                "tab:orange",
                "tab:olive",
                "tab:gray",
                "tab:brown",
               ]

list_line_styles = ["solid","dotted","dashed","dashdot","solid","dotted","dashed","dashdot"]

In [ ]:
fig1, ax1 = plt.subplots(figsize=aspect_ratio, dpi=200)
fig2, ax2 = plt.subplots(figsize=aspect_ratio, dpi=200)

mean_pressure_estimate = 0.0
mean_pressure = 0.0
mean_difference = 0.0

idir = 0
time = multi_data["time"] / 1.0e-9
surface_area = multi_data["surface_area"]
intensity_14 = multi_data["intensity"] / 1.0e14
pressure_estimate = multi_data["pressure_estimate"] # Mbars

peak_convergence = multi_data["tind_max_rhor_DT"]

t_lims = np.array([0.0, 0.0])
t_lims[0] = np.min(time[1:]) #/ time[peak_convergence])
t_lims[1] = np.max(time[1:]) #/ time[peak_convergence])
intensity_threshold = 351.0**2 / (multi_data["wavelength"] * 1.0e9)**2
max_intensity = np.max(intensity_14 / 10.0)
print("Ratio of max intensity to LPI threshold for is: {:.2f}".format(max_intensity / intensity_threshold))

ablation_pressure = np.zeros(multi_data["ntimes"])
for tind in range(multi_data["ntimes"]):
    ablation_pressure[tind] = multi_data["PT"][tind,multi_data["ind_outer_surf"][tind]]

plot_time = 12.0
tind = np.argmin(np.abs(multi_data["time"][:] - plot_time * 1.0e-9))

ind_max_abl_pressure = np.argmax(pressure_estimate)
print(multi_data["time"][ind_max_abl_pressure])
print("Max ablation pressure estimate is: {:.2f} Mbars".format(np.max(pressure_estimate)))
mean_pressure_estimate += np.max(pressure_estimate)
print("Max pressure at ablation front is: {:.2f} Mbars".format(ablation_pressure[ind_max_abl_pressure] / 1.0e12))
mean_pressure += ablation_pressure[ind_max_abl_pressure] / 1.0e12
print("Difference, is: {:.2f} Mbars".format(np.abs(np.max(pressure_estimate) - ablation_pressure[ind_max_abl_pressure] / 1.0e12)))
mean_difference += np.abs(np.max(pressure_estimate) - ablation_pressure[ind_max_abl_pressure] / 1.0e12)**2

ax1.plot(time[1:], intensity_14 / 10.0, color=list_colours[idir], linestyle=list_line_styles[0], linewidth=3)
ax1.plot(t_lims, np.array([1.0, 1.0]) * intensity_threshold, label="LPI threshold", color=list_colours[idir], linestyle=list_line_styles[1])
ax2.plot(time[1:], pressure_estimate, color=list_colours[idir], label=r"$P_{abl}$ estimate", linestyle=list_line_styles[0], linewidth=5) #/ time[peak_convergence]
#ax2.plot(time[1:] , ablation_pressure[1:] / 1.0e12, label=r"$P_{abl}$ hydro", color=list_colours[idir], linestyle=list_line_styles[1], linewidth=3) #/ time[peak_convergence]
#ax2.plot(time[1:] / time[peak_convergence], np.max(multi_data["PT"][1:],axis=1) / 1.0e12, label=" ", color=list_colours[idir], linestyle=list_line_styles[1], linewidth=3)
#ax2.plot(dataset_params["plasma_profile_times"][:],dataset["avg_flux"][ind_max_fitness_temp[3],:], "x:")

count = 1
for ind in (ind_max_fitness,ind_max_fitness_temp[:]):
    ax2.plot(dataset_params["plasma_profile_times"],dataset["avg_flux"][ind,:], "X:", label=labels[ind], markersize=20, linewidth=5, color=list_colours[count])
    count += 1

xlims = [0.,16.0]

#ax1.set_xlim(xlims)
ax1.set_ylabel(r"Convergent intensity ($10^{15}$)")
ax1.set_xlabel("Time (ns)")
ax1.legend(loc="upper left")
#fig1.savefig(root+"/"+plot_dir + "/" + 'hydro_equivalent_intensity.png', bbox_inches='tight')
fig2.savefig(diag_dir+"/time_dependant_intensity_with_mult_outputs" + sys_params["plot_file_type"], dpi=300, bbox_inches='tight')

ylims = [0.,300]
#ax2.set_xlim(xlims)
ax2.set_ylim(ylims)
ax2.set_xlabel("Time (ns)")
ax2.set_ylabel("Ablation pressure (Mbars)")
ax2.legend(loc="upper left")
fig2.savefig(diag_dir+"/time_dependant_pabl_with_mult_outputs" + str(tind) + sys_params["plot_file_type"], dpi=300, bbox_inches='tight')
#fig2.savefig(root+"/"+plot_dir + "/" + 'hydro_equivalent_ablation_pressure_estimate.png', bbox_inches='tight')

# Compare illuminations

In [ ]:
list_labels = ["No CBET","CBET","CBET + 1% Bandwidth"]
list_dirs = ["../Data/250521d_527nm_5MJ_cpm48_plasma", "../Data/250521c_527nm_5MJ_cpm48_plasma_cbet", "../Data/250521b_527nm_5MJ_cpm48_plasma_bandwidth"]
ndirs = len(list_dirs)


dataset = {}
dataset_params = {}
deck_gen_params = {}
facility_spec = {}
sys_params = {}
opt_params = {}
fitness_temporal = {}
fitness_overall = {}

for idir in range(ndirs):
    label = list_labels[idir]
    diag_dir = list_dirs[idir]
    
    sys_params[label] = tdg.define_system_params(diag_dir)
    dataset[label], dataset_params[label], deck_gen_params[label], facility_spec[label] = idg.load_data_dicts_from_file(sys_params[label])
    opt_params[label] = uopt.define_optimizer_parameters(diag_dir, 0, 0, dataset_params[label], facility_spec[label], sys_params[label])
    
    fitness_temporal[label] = fitness_function_time_dependant(dataset[label], opt_params[label])
    fitness_overall[label] = fitness_function(dataset[label], opt_params[label])

In [ ]:
fig1, ax1 = plt.subplots(figsize=aspect_ratio, dpi=200)
fig2, ax2 = plt.subplots(figsize=aspect_ratio, dpi=200)

mean_pressure_estimate = 0.0
mean_pressure = 0.0
mean_difference = 0.0

idir = 0
time = multi_data["time"] / 1.0e-9
surface_area = multi_data["surface_area"]
intensity_14 = multi_data["intensity"] / 1.0e14
pressure_estimate = multi_data["pressure_estimate"] # Mbars

peak_convergence = multi_data["tind_max_rhor_DT"]

t_lims = np.array([0.0, 0.0])
t_lims[0] = np.min(time[1:]) #/ time[peak_convergence])
t_lims[1] = np.max(time[1:]) #/ time[peak_convergence])
intensity_threshold = 351.0**2 / (multi_data["wavelength"] * 1.0e9)**2
max_intensity = np.max(intensity_14 / 10.0)
print("Ratio of max intensity to LPI threshold for is: {:.2f}".format(max_intensity / intensity_threshold))

ablation_pressure = np.zeros(multi_data["ntimes"])
for tind in range(multi_data["ntimes"]):
    ablation_pressure[tind] = multi_data["PT"][tind,multi_data["ind_outer_surf"][tind]]

plot_time = 12.0
tind = np.argmin(np.abs(multi_data["time"][:] - plot_time * 1.0e-9))

ind_max_abl_pressure = np.argmax(pressure_estimate)
print(multi_data["time"][ind_max_abl_pressure])
print("Max ablation pressure estimate is: {:.2f} Mbars".format(np.max(pressure_estimate)))
mean_pressure_estimate += np.max(pressure_estimate)
print("Max pressure at ablation front is: {:.2f} Mbars".format(ablation_pressure[ind_max_abl_pressure] / 1.0e12))
mean_pressure += ablation_pressure[ind_max_abl_pressure] / 1.0e12
print("Difference, is: {:.2f} Mbars".format(np.abs(np.max(pressure_estimate) - ablation_pressure[ind_max_abl_pressure] / 1.0e12)))
mean_difference += np.abs(np.max(pressure_estimate) - ablation_pressure[ind_max_abl_pressure] / 1.0e12)**2

ax1.plot(time[1:], intensity_14 / 10.0, color=list_colours[idir], linestyle=list_line_styles[0], linewidth=3)
ax1.plot(t_lims, np.array([1.0, 1.0]) * intensity_threshold, label="LPI threshold", color=list_colours[idir], linestyle=list_line_styles[1])
ax2.plot(time[1:], pressure_estimate, color=list_colours[idir], label=r"$P_{abl}$ estimate", linestyle=list_line_styles[0], linewidth=3) #/ time[peak_convergence]
ax2.plot(time[1:] , ablation_pressure[1:] / 1.0e12, label=r"$P_{abl}$ hydro", color=list_colours[idir], linestyle=list_line_styles[1], linewidth=3) #/ time[peak_convergence]
#ax2.plot(time[1:] / time[peak_convergence], np.max(multi_data["PT"][1:],axis=1) / 1.0e12, label=" ", color=list_colours[idir], linestyle=list_line_styles[1], linewidth=3)
#ax2.plot(dataset_params["plasma_profile_times"][:],dataset["avg_flux"][ind_max_fitness_temp[3],:], "x:")
"""
for ind in ind_max_fitness_temp:
    ax2.plot(dataset_params["plasma_profile_times"][:],dataset["avg_flux"][ind,:], "x:")#, label=labels[ind])
"""

plot_index = 7
for idir in range(ndirs):
    label = list_labels[idir]
    diag_dir = list_dirs[idir]
    ax2.plot(dataset_params[label]["plasma_profile_times"][:],dataset[label]["avg_flux"][plot_index,:], "x:", label=label, color=list_colours[idir+1])

xlims = [0.,1.1]

#ax1.set_xlim(xlims)
ax1.set_ylabel(r"Convergent intensity ($10^{15}$)")
ax1.set_xlabel("Time (ns)")
ax1.legend(loc="upper left")
#fig2.savefig(diag_dir+"/time_dependant_intensity_with_mult_outputs" + sys_params["plot_file_type"], dpi=300, bbox_inches='tight')
fig1.savefig(sys_params[label]["root_dir"]+"/"+sys_params[label]["figure_location"]+"/time_dependant_intensity_with_mult_outputs_comparison" + sys_params[label]["plot_file_type"], dpi=300, bbox_inches='tight')

ylims = [0.,300]

#ax2.set_xlim(xlims)
ax2.set_ylim(ylims)
ax2.set_xlabel("Time (ns)")
ax2.set_ylabel("Ablation pressure (Mbars)")
ax2.legend(loc="upper left")
fig2.savefig(sys_params[label]["root_dir"]+"/"+sys_params[label]["figure_location"]+"/time_dependant_pabl_with_mult_outputs_comparison" + sys_params[label]["plot_file_type"], dpi=300, bbox_inches='tight')